In [1]:
import numpy as np
import pandas as pd

import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Import dataframe from pickle
df = pd.read_pickle('dataset/dataframe.pickle', compression='gzip')

## Issue Date Time

In [3]:
## Count tickets per year
df.issue_date.dt.year.value_counts()

2015.0    2181331
2016.0    1783038
2014.0      39406
2017.0      33936
2013.0         38
2012.0         30
2010.0         12
2011.0         10
Name: issue_date, dtype: int64

In [4]:
## Only 2015 and 2016 has complete dataset so filter those out
df = df[df.issue_date.between('2015', '2017')]

In [5]:
## There are only a few nan values for issue_time
print('Number of nan Issue time: {}'.format((df.issue_time == -1).sum()))
## Replace -1 with median time because the issue time is pretty normal
df.loc[df.issue_time == -1, 'issue_time'] = df.issue_time.median()

Number of nan Issue time: 844


In [6]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    print(df.issue_time.describe())

count   3964369
mean       1230
std         469
min           0
25%         927
50%        1208
75%        1546
max        2359
Name: issue_time, dtype: float64


In [7]:
## Combine Issue Date and Issue Time 
## Convert time to int to remove decimal then to str to pad the front with zero 
## Final format HHMM
time = df.issue_time.astype('int').astype('str').str.pad(width = 4, side = 'left', fillchar = '0')
## Combine date and time
date_time = df.issue_date.astype('str') + ' ' + time
## Add to dataframe
df['issue_datetime'] = pd.to_datetime(date_time, format = '%Y-%m-%d %H%M')

In [8]:
## Verify conversion was correct
df[['issue_datetime', 'issue_date', 'issue_time']].head()

,issue_datetime,issue_date,issue_time
0,2015-12-30 22:01:00,2015-12-30,2201.0
1,2015-12-30 22:05:00,2015-12-30,2205.0
2,2015-12-30 17:25:00,2015-12-30,1725.0
3,2015-12-30 17:38:00,2015-12-30,1738.0
4,2015-12-30 18:07:00,2015-12-30,1807.0


## Fine Amount

In [9]:
## Impute fine_amount 
with pd.option_context('display.float_format', '{:.0f}'.format):
    print(df.fine_amount.describe())
print('Number of nan: {}'.format((df.fine_amount == -1).sum()))

count   3964369
mean         70
std          32
min          -1
25%          63
50%          68
75%          73
max         505
Name: fine_amount, dtype: float64
Number of nan: 1532


In [10]:
## Fine looks a little skewed so use median
df.loc[df.fine_amount == -1, 'fine_amount'] = df.fine_amount.median()
print('Verify that there are no more nan: {}'.format((df.fine_amount == -1).sum()))

Verify that there are no more nan: 0


In [11]:
## After imputation, I found that a few of them are $10
## This is incorrect because the minimum fine is 25
## for Display of tabs
print('Fine amount less than $25: {}'.format((df.fine_amount < 25).sum()))
## All of the violations are Display of tabs
print('Number of violations for Display of tabs: {}'.\
      format((df.loc[df.fine_amount < 25, 'violation_description'] == 'DISPLAY OF TABS').sum()))
## From lookup_v_fine.csv, that fine amount is $25
df.loc[df.fine_amount < 25, 'fine_amount'] = 25
## Verify fine correction
print('Fine amount less than $25 after correction: {}'.format((df.fine_amount < 25).sum()))

Fine amount less than $25: 25
Number of violations for Display of tabs: 25
Fine amount less than $25 after correction: 0


In [12]:
## Downcast fine_amount 
df['fine_amount'] = pd.to_numeric(df.fine_amount, downcast = 'unsigned')

## Drop columns

In [13]:
## Remove columns that aren't useful
df.drop(columns= ['meter_id', 'marked_time', 
                  'plate_expiry_date', 'vin', 
                  'body_style', 'color', 
                  'route', 'violation_code', 
                  'latitude', 'longitude', 
                  'issue_date', 'issue_time'], errors = 'ignore', inplace = True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3964369 entries, 0 to 4004294
Data columns (total 8 columns):
ticket_number            uint64
rp_state_plate           object
make                     object
location                 object
agency                   category
violation_description    object
fine_amount              uint16
issue_datetime           datetime64[ns]
dtypes: category(1), datetime64[ns](1), object(4), uint16(1), uint64(1)
memory usage: 223.1+ MB


## Export dataframe to csv

In [142]:
df.to_csv('dataset/la_citation_tableau.csv', index = False)

## Count Citations By Block

https://egis3.lacounty.gov/eGIS/wp-content/uploads/2013/10/House-Numbering-and-Street-Naming-Material-2013.pdf

This article says that generally blocks are every 100 

In [16]:
## Get only the year
df['year'] = df.issue_datetime.dt.year

In [31]:
## Count citations per address
street_citations = df.groupby(['year', 'location'])

In [32]:
street_citations = street_citations.agg({'ticket_number' : 'count', 'fine_amount': 'sum'})

In [40]:
street_citations.reset_index(inplace = True)

In [64]:
print(street_citations.shape)
street_citations.sort_values(by= 'ticket_number', ascending=False, inplace=True)
street_citations.head(10)

(1161941, 4)


,year,location,ticket_number,fine_amount
60870,2015,11600 SAN VICENTE BL,2579,154224.0
84864,2015,1235 FIGUEROA PL,2468,140734.0
708260,2016,1301 ELECTRIC AVE,2062,124784.0
5907,2015,101 LARCHMONT BL N,1966,120476.0
672023,2016,11600 SAN VICENTE BL,1936,116816.0
76839,2015,12100 VENTURA BL,1494,93902.0
854193,2016,2377 MIDVALE AVE,1458,90146.0
64066,2015,11700 SAN VICENTE BL,1422,85762.0
60981,2015,11601 SAN VICENTE BL,1397,83161.0
170060,2015,1608 PACIFIC AVE S,1370,83887.0


Can see that 11600 San Vicente Bl and 11601 San Vicente Bl should be together since they are on the same block. 

In [98]:
## Split location by first space character which is usually the street number
tmp = street_citations.location.str.partition(' ')
## Get index where location has correct street numbering
with_streetnumber = tmp[0].str.isdigit()
tmp = tmp[with_streetnumber] ## Remove incorrect addressing
## Convert to integer and floor integer to its nearest hundred place
tmp[0] = (pd.to_numeric(tmp[0])//100 * 100).astype('str')

In [100]:
## Remove incorrect addressing
street_citations = street_citations[with_streetnumber]

In [101]:
street_citations['block'] = tmp[0] + ' ' + tmp[2]

In [102]:
street_citations.head(10)

,year,location,ticket_number,fine_amount,block
60870,2015,11600 SAN VICENTE BL,2579,154224.0,11600 SAN VICENTE BL
84864,2015,1235 FIGUEROA PL,2468,140734.0,1200 FIGUEROA PL
708260,2016,1301 ELECTRIC AVE,2062,124784.0,1300 ELECTRIC AVE
5907,2015,101 LARCHMONT BL N,1966,120476.0,100 LARCHMONT BL N
672023,2016,11600 SAN VICENTE BL,1936,116816.0,11600 SAN VICENTE BL
76839,2015,12100 VENTURA BL,1494,93902.0,12100 VENTURA BL
854193,2016,2377 MIDVALE AVE,1458,90146.0,2300 MIDVALE AVE
64066,2015,11700 SAN VICENTE BL,1422,85762.0,11700 SAN VICENTE BL
60981,2015,11601 SAN VICENTE BL,1397,83161.0,11600 SAN VICENTE BL
170060,2015,1608 PACIFIC AVE S,1370,83887.0,1600 PACIFIC AVE S


In [105]:
## Group by block
block_citation = street_citations\
    .groupby(['year', 'block'])\
    .agg({'ticket_number': 'sum', 'fine_amount': 'sum'})\
    .sort_values(by = 'ticket_number', ascending = False).reset_index()
block_citation.head()

,year,block,ticket_number,fine_amount
0,2015,0 WASHINGTON BLVD E,5491,334621.0
1,2015,11600 SAN VICENTE BL,5094,304131.0
2,2015,100 LARCHMONT BL N,5094,308442.0
3,2016,11600 SAN VICENTE BL,3792,228214.0
4,2015,700 WALL ST S,3729,223087.0


In [106]:
block_citation['fine_amount'] = pd.to_numeric(block_citation.fine_amount, downcast='unsigned')

In [109]:
block_citation.columns = ['year', 'block', 'number_of_citations', 'revenue']

In [111]:
block_citation.head()

,year,block,number_of_citations,revenue
0,2015,0 WASHINGTON BLVD E,5491,334621
1,2015,11600 SAN VICENTE BL,5094,304131
2,2015,100 LARCHMONT BL N,5094,308442
3,2016,11600 SAN VICENTE BL,3792,228214
4,2015,700 WALL ST S,3729,223087


In [112]:
block_citation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248838 entries, 0 to 248837
Data columns (total 4 columns):
year                   248838 non-null int64
block                  248838 non-null object
number_of_citations    248838 non-null int64
revenue                248838 non-null uint32
dtypes: int64(2), object(1), uint32(1)
memory usage: 6.6+ MB


In [129]:
block_citation.to_csv('dataset/block_citation.csv', index = False)

## Top Block Citation Plus there GPS coordinates

In [116]:
## Dataframe is pre-sorted
## Get top 10 for each year
top_block = pd.concat([block_citation.query('year == 2015').head(10),
                       block_citation.query('year == 2016').head(10)], ignore_index=True)

In [136]:
## Manually search on google map
## Reason because address can be ambigous 
## like 11700 Barrington CT S has multiple options:
## South Barrington or Barrington Court
gps = [['0 WASHINGTON BLVD E', 34.031056, -118.264450],
       ['11600 SAN VICENTE BL', 34.053524, -118.461610],
       ['100 LARCHMONT BL N', 34.073213, -118.323628],
       ['700 WALL ST S', 34.041417, -118.249039],
       ['0 WINDWARD AVE E', 33.987279, -118.472977],
       ['7000 HAWTHORN AVE', 34.100360, -118.342598],
       ['11700 SAN VICENTE BL', 34.053356, -118.466455],
       ['12100 VENTURA BL', 34.143420, -118.396918],
       ['300 2ND ST E', 34.048428, -118.240440],
       ['11700 BARRINGTON CT S', 34.064209, -118.469739],
       ['11600 SAN VICENTE BL', 34.053524, -118.461610],
       ['7000 HAWTHORN AVE', 34.100360, -118.342598],
       ['100 LARCHMONT BL N', 34.073213, -118.323628],
       ['700 WALL ST S', 34.041417, -118.249039],
       ['0 WASHINGTON BLVD E', 34.031056, -118.264450],
       ['1600 ORANGE DR N', 34.100784, -118.341414],
       ['11700 SAN VICENTE BL', 34.053356, -118.466455],
       ['1300 ELECTRIC AVE', 33.991233, -118.466241],
       ['12100 VENTURA BL', 34.143420, -118.396918],
       ['2300 MIDVALE AVE', 34.040915, -118.430583]]
gps_df = pd.DataFrame(gps, columns = ['loc', 'lat', 'long'])

In [137]:
top_block['latitude'] = gps_df.lat
top_block['longitude'] = gps_df.long

In [138]:
top_block

,year,block,number_of_citations,revenue,latitude,longitude
0,2015,0 WASHINGTON BLVD E,5491,334621,34.031056,-118.264450
1,2015,11600 SAN VICENTE BL,5094,304131,34.053524,-118.461610
2,2015,100 LARCHMONT BL N,5094,308442,34.073213,-118.323628
3,2015,700 WALL ST S,3729,223087,34.041417,-118.249039
4,2015,0 WINDWARD AVE E,3577,223083,33.987279,-118.472977
5,2015,7000 HAWTHORN AVE,3523,240308,34.100360,-118.342598
6,2015,11700 SAN VICENTE BL,3162,191433,34.053356,-118.466455
7,2015,12100 VENTURA BL,3010,190039,34.143420,-118.396918
8,2015,300 2ND ST E,2802,184124,34.048428,-118.240440
9,2015,11700 BARRINGTON CT S,2632,158913,34.064209,-118.469739


In [139]:
top_block.to_csv('dataset/top_block_gps.csv', index=False)